In [1]:
import torch, os, re, random
import random, time, os, pytz, argparse, yaml, sys
#sys.path.append("../scripts/probing")
#sys.path.append("../scripts")
os.chdir("../scripts/probing")
sys.path.append("../")
import numpy as np
from probing.probe import *
from tqdm import tqdm, trange
from probing_utils import *
from torch import nn
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [63]:
handle = "0921_104304"
with open(f"ckpts/{handle}/configs.yaml", 'rb') as f: config = yaml.full_load(f.read()) # dict
LM = __import__(config["lm"])
tokenizer = LM.get_tokenizer(**config["lm_kwargs"])
config["lm_kwargs"]["token_pos"] = ""

model = PROBE(
    config['hidden_dim_multipliers'], 
    len(config['relations'].split()), 
    config["lm"],
    config["lm_kwargs"]
).to(device)
model.load_state_dict(torch.load(f"ckpts/{handle}/model.pt"))

criterion = nn.CrossEntropyLoss().to(device)

In [68]:
texts = ["a balloon is behind a cup"]
input_ids = tokenizer.batch_encode_plus(
        texts,
        return_tensors = "pt",
        padding = True,
        truncation = True
    ).input_ids

encoded_text = model.encode_subj_obj(texts, [],[])

In [69]:
x = encoded_text.view(-1, model.text_embed_dim) # batch_size*2, dim
x = model.out(model.layers(x)).transpose(0,1)
print(x.size()) # batch_size, num_classes
tokens = tokenizer.convert_ids_to_tokens(input_ids.tolist()[0])
#print("\n{:16s} {:7s} {:7s}".format("token", "r", "!r"))
#for tok, r, r_ in zip(tokens, x[0], x[1]):
#    print("{:16s} {:7.4f} {:7.4f}".format(tok, r.item(), r_.item()))
print("\n{:16s} {:7s} {:7s} {:7s} {:7s} {:7s} {:7s}".format("token", "top", "bottom", "front", "behind", "inside", "outside"))
for tok, r0, r1, r2, r3, r4, r5 in zip(tokens, x[0], x[1], x[2], x[3], x[4], x[5]):
    print("{:16s} {:7.4f} {:7.4f} {:7.4f} {:7.4f} {:7.4f} {:7.4f}".format(tok, r0.item(), r1.item(), r2.item(), r3.item(), r4.item(), r5.item()))

torch.Size([6, 8])

token            top     bottom  front   behind  inside  outside
<|startoftext|>  -0.4917 -0.8844  1.7860  0.0201 -0.4175 -0.1509
a</w>             0.1109 -0.4813 -1.1418 -1.2865  2.9243 -0.3061
balloon</w>       0.2821  0.3111 -0.2777 -0.2458 -0.0551  0.1729
is</w>           -1.1601 -1.4428  3.5731 -0.5875  0.4735 -1.1746
behind</w>        0.3331 -3.9804 10.0252 -3.4241 -1.1613 -2.3603
a</w>            -0.6350 -0.8396  5.9970 -3.0409  1.3270 -3.5555
cup</w>           0.6243 -4.2512 11.6778 -2.2578 -2.8801 -4.6556
<|endoftext|>     1.5570 -3.2904  5.4882 -0.1057  0.1612 -5.4092


#### Does CLIP send information about the correct **relation pair** to nouns?

In [76]:
handle = "0921_104117"
with open(f"ckpts/{handle}/configs.yaml", 'rb') as f: config = yaml.full_load(f.read()) # dict
LM = __import__(config["lm"])
tokenizer = LM.get_tokenizer(**config["lm_kwargs"])

model = PROBE(
    config['hidden_dim_multipliers'], 
    len(config['relations'].split()), 
    config["lm"],
    config["lm_kwargs"]
).to(device)
model.load_state_dict(torch.load(f"ckpts/{handle}/model.pt"))

<All keys matched successfully>

In [77]:
ood_test_data = Texts(
    relations=config['relations'].split(),
    nouns_file=config['ood_nouns_file'],
)
dataloader = DataLoader(ood_test_data, shuffle=True, batch_size=config['batch_size'])


In [78]:
lazy_accs = []
for (texts, labels) in tqdm(dataloader, desc="Testing"):
    output = model(list(texts)) # batch_size*2, num_classes
    gth = torch.cat([labels, 1 - (labels % 2) + 2*(labels // 2)], dim=1).long().view((-1,)).to(device)
    pred = output.argmax(1, keepdim=True)
    print(texts)
    print(pred.squeeze())
    print(gth)
    lazy_accs.append(get_lazy_acc(output, gth).item())
    break
print(len(lazy_accs), np.mean(lazy_accs))
    

Testing:   0%|          | 0/82 [00:00<?, ?it/s]

('a towel is at the bottom of a container', 'a blouse is at the bottom of a hook', 'a calender is outside of a hourglass', 'a saddle is outside of a trumpet', 'a miniature is in front of a container', 'a postcard is on top of a screen', 'a jar is outside of a container', 'a hourglass is on top of a trumpet', 'a dresser is on top of a brownie', 'a hourglass is behind a camera', 'a brownie is inside a face', 'a stone is in front of a camera', 'a jar is outside of a scale', 'a face is at the bottom of a container', 'a plant is behind a face', 'a blouse is behind a face', 'a face is in front of a calender', 'a diary is in front of a screen', 'a dryer is outside of a bathtub', 'a bathtub is at the bottom of a scale', 'a bathtub is outside of a rock', 'a dryer is inside a postcard', 'a camera is inside a cable', 'a camera is outside of a head', 'a cable is outside of a scale', 'a dresser is inside a backpack', 'a bathtub is behind a scale', 'a card is inside a calender', 'a miniature is insi